# Noise2Void on 3D Patches applied to Refined Dataset

In this experiment we trained a model on the REFINED train set, using no augmentations and a 3D patch size of 64x64x64. 

The following command was used to train the model:

```bash
python n2v_train.py 
-e .env_hpc 
--level DEBUG 
--train_dataset_name=train 
--validation_dataset_name=val 
--DATASET_FOLDER=/scratch/edoardo.giacomello/calcium_imaging/refined 
--experiment_name=REFINED_N2V_3 
--patch_size_z=64 
--batch_size=1 
--axes=ZYX
```

Predictions were made on the train set to enable further models to be trained on the predictions. The following command was used to make predictions:

```bash
python n2v_predict.py
 -e .env_hpc 
 --level DEBUG 
 --dataset_name=train 
 --DATASET_FOLDER=/scratch/edoardo.giacomello/calcium_imaging/refined 
 --experiment_name=REFINED_N2V_3 
 --patch_size_z=64 
 --batch_size=4 
 --axes=ZYX
```

# Model Output

In [ ]:
import tifffile
import matplotlib.pyplot as plt

import ipywidgets as widgets
from IPython.display import display

def display_image_with_slider(image):
    T, Y, X = image.shape
    vmin = image.min()
    vmax = image.max()

    def update_image(t):
        plt.figure(figsize=(10, 10))  # Set the figure size
        plt.imshow(image[t], cmap='gray', vmin=vmin, vmax=vmax)
        plt.show()

    slider = widgets.IntSlider(min=0, max=T-1, step=1, description='T')
    widgets.interact(update_image, t=slider)


EXAMPLE_TIFF = "output/REFINED_N2V_3/20ms_20%_Yoda1_005.tif"

example_image = tifffile.imread(EXAMPLE_TIFF)
display_image_with_slider(example_image)


interactive(children=(IntSlider(value=0, description='T', max=1008), Output()), _dom_classes=('widget-interact…

# Subtract the average noise from the microscope

In [ ]:
background_noise = tifffile.imread("/localscratch/calcium_imaging_dataset/calcium_imaging/average_image.tif")
mean_subtracted = example_image - background_noise
display_image_with_slider(mean_subtracted)

interactive(children=(IntSlider(value=0, description='T', max=1008), Output()), _dom_classes=('widget-interact…